<a href="https://colab.research.google.com/github/ko4ro/Katakana/blob/master/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import codecs as cd
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
import seaborn as sns

In [7]:
!nvidia-smi

Sat May 29 09:11:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
from google.colab import files

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

!kaggle kernels list — user YOUR_USER — sort-by dateRun

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [9]:
df_train = pd.read_csv('/content/df_train.csv')
#最初の五行を表示してみる。
display(df_train.head())

,ID,ACTIVITY,TIME,SL,EEG,BP,HR,CIRCLUATION
0,0,4,1954.23,55.5372,-368.000,97,33,5
1,1,0,17450.00,120164.0000,-7539.000,66,347,4535
2,2,0,12024.50,19634.0000,-3361.000,86,167,973
3,3,4,13413.00,46643.3000,-5735.430,65,273,2061
4,4,2,5325.98,3491.2600,-809.996,50,51,132


In [13]:
df_train_label = df_train['ACTIVITY']
df_train = df_train.drop(columns=['ID','ACTIVITY'])

In [46]:
df_train_label.head()

0    4
1    0
2    0
3    4
4    2
Name: ACTIVITY, dtype: int64

In [47]:
df_train.head()

,TIME,SL,EEG,BP,HR,CIRCLUATION,DAYTIME
0,2,55.5372,-368.000,97,33,5,morning
1,2,120164.0000,-7539.000,66,347,4535,morning
2,2,19634.0000,-3361.000,86,167,973,morning
3,2,46643.3000,-5735.430,65,273,2061,morning
4,2,3491.2600,-809.996,50,51,132,morning


In [33]:
def get_daytime(td):
    m, s = divmod(td.seconds, 60)
    h, m = divmod(m, 60)
    if 4 < h < 6:
      return "dawn"
    elif h < 12:
      return "morning"
    elif h < 16:
      return "afternoon"
    elif h < 16:
      return "evening"
    else:
      return "night"

In [39]:
import datetime
datimes = [ get_daytime(datetime.timedelta(seconds=second)) for second in df_train["TIME"]]
df_train['TIME'] = datimes

In [44]:
df_train.head()

,TIME,SL,EEG,BP,HR,CIRCLUATION,DAYTIME
0,2,55.5372,-368.000,97,33,5,morning
1,2,120164.0000,-7539.000,66,347,4535,morning
2,2,19634.0000,-3361.000,86,167,973,morning
3,2,46643.3000,-5735.430,65,273,2061,morning
4,2,3491.2600,-809.996,50,51,132,morning


In [41]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le = le.fit(df_train['TIME'])
df_train['TIME'] = le.transform(df_train['TIME'])

In [43]:
#統計値を表示してみる。
display(df_train.describe())
#欠損値がある行数を数えてみる。
display(df_train.isnull().sum())
#各列のデータタイプを確認してみる。
display(df_train.dtypes)

,TIME,SL,EEG,BP,HR,CIRCLUATION
count,14744.000000,1.474400e+04,1.474400e+04,14744.000000,14744.000000,14744.000000
mean,1.946487,7.558490e+04,-5.779018e+03,58.437670,211.658098,2901.420510
std,0.226265,1.290485e+05,1.140682e+05,48.277632,130.502892,3855.159777
min,0.000000,4.222420e+01,-1.262600e+07,0.000000,33.000000,5.000000
25%,2.000000,9.917505e+03,-5.640000e+03,25.000000,119.000000,587.000000
50%,2.000000,3.116200e+04,-3.363255e+03,45.000000,180.000000,1581.000000
75%,2.000000,8.070895e+04,-2.155435e+03,79.000000,271.000000,3539.000000
max,2.000000,2.426140e+06,1.410000e+06,506.000000,986.000000,52210.000000


TIME           0
SL             0
EEG            0
BP             0
HR             0
CIRCLUATION    0
DAYTIME        0
dtype: int64

TIME             int64
SL             float64
EEG            float64
BP               int64
HR               int64
CIRCLUATION      int64
DAYTIME         object
dtype: object

In [ ]:
df_train.to_csv('traindata.csv')
df_train_label.to_csv('trainlabel.csv')

In [49]:
# LassoCVを使って、正則化の強さを自動決定したモデル（estimator）を作成
estimator = LassoCV(normalize=True, cv=10)

# モデルの情報を使って特徴選択を行うためSelectFromModelを使う
# 今回は係数がしきい値（threshold）が1e-5以下である特徴を削除する
sfm = SelectFromModel(estimator, threshold=1e-5)

# fitで特徴選択を実行
sfm.fit(df_train, df_train_label)

SelectFromModel(estimator=LassoCV(alphas=None, copy_X=True, cv=10, eps=0.001,
                                  fit_intercept=True, max_iter=1000,
                                  n_alphas=100, n_jobs=None, normalize=True,
                                  positive=False, precompute='auto',
                                  random_state=None, selection='cyclic',
                                  tol=0.0001, verbose=False),
                max_features=None, norm_order=1, prefit=False, threshold=1e-05)

In [50]:
# 削除すべき特徴の名前を取得 
removed_idx  = ~sfm.get_support()
df_train.columns[removed_idx] 

Index(['SL', 'EEG'], dtype='object')